In [ ]:
%pip install azure-cosmos
%pip install faker

In [4]:
from azure.cosmos import CosmosClient, PartitionKey
from configparser import ConfigParser
from faker import Faker
from collections import OrderedDict

import os
import json
import uuid
import datetime


parser = ConfigParser()
parser.read('../NotebookConfig.cfg')

cosmosAccountURI = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_URI')
cosmosAccountKey = parser.get('CosmosDB', 'COSMOSDB_ACCOUNT_KEY')

databaseName = 'Learn'
containerName = 'AISearch'
partitionKeypath = '/PartitionKey'

Faker.seed(0)
fake = Faker(['en-US'])

print(cosmosAccountURI)

https://cosmicgbbcdb-sql.documents.azure.com:443/


In [9]:
client = CosmosClient(cosmosAccountURI, cosmosAccountKey)
db = client.create_database_if_not_exists(databaseName)

pkPath = PartitionKey(path=partitionKeypath)
ctr = db.create_container_if_not_exists(id=containerName, partition_key=pkPath, offer_throughput=400, default_ttl=-1) 

# Default TTL is set to -1, which means the documents will never expire unless you set a value in seconds

In [10]:
maxrange = 100

for j in range(maxrange):
    # readings = []

    # *** Produce N readings
    # Execute this repeatedly by changing range from 1, 100, 1000 and 10000
    # for r in range(1):
    #     readings.append(
    #         {
    #             'Dimension': fake.random_element(elements=('D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'))
    #             , 'Value': fake.random_number(digits=5)
    #             , 'Timestamp': fake.date_time_this_year().isoformat()
    #         })

    docId = str(uuid.uuid4())

    doc = {
        'id': docId
        , 'PartitionKey': docId
        , 'Name': fake.name()
        , 'Title': fake.sentence(nb_words=6, variable_nb_words=True, ext_word_list=None)
        , 'Description': fake.sentence(nb_words=100, variable_nb_words=True, ext_word_list=None)
        , 'Class': fake.random_element(elements=OrderedDict([("A", 0.40), ("B", 0.35), ("C", 0.15), ("D", 0.05), ("E", 0.05)]))
        , 'UpdateTS': datetime.datetime.now().timestamp()
        
    }
    
    ctr.create_item(doc)

print('Completed writing ' + str(maxrange) + ' documents to CosmosDB')

TypeError: can only concatenate str (not "int") to str

Clean up code

In [ ]:
# Assume objects are instantiated
db.delete_container(containerName)

# import glob

# files = glob.glob('.\OutputFiles\*')
# for f in files:
#     os.remove(f)